# 📅 Test DateTime MCP Server con Agent SDK

Este notebook demuestra cómo usar el MCP Server de fecha/hora con el framework de OpenAI Agents.

## 🔧 Setup

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

## 📋 Listar Herramientas Disponibles

In [2]:
# Configure the datetime server
params = {"command": "uv", "args": ["run", "datetime_server.py"]}

# List available tools
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

print("📋 Available DateTime Tools:")
print("="*60)
for tool in mcp_tools:
    print(f"\n🔧 {tool.name}")
    print(f"   {tool.description}")
    print(f"   Parameters: {list(tool.inputSchema.get('properties', {}).keys())}")

📋 Available DateTime Tools:

🔧 get_current_date
   
    Get the current date in YYYY-MM-DD format.
    
    Returns:
        Current date as a string (e.g., '2026-02-04')
    
   Parameters: []

🔧 get_current_time
   
    Get the current time in HH:MM:SS format.
    
    Returns:
        Current time as a string (e.g., '14:30:45')
    
   Parameters: []

🔧 get_current_datetime
   
    Get the current date and time in a human-readable format.
    
    Returns:
        Current datetime as a string (e.g., 'Tuesday, February 04, 2026 at 14:30:45')
    
   Parameters: []

🔧 get_timestamp
   
    Get the current Unix timestamp.
    
    Returns:
        Unix timestamp as a string (seconds since epoch)
    
   Parameters: []

🔧 get_datetime_in_timezone
   
    Get the current date and time in a specific timezone.
    
    Args:
        timezone: Timezone name (e.g., 'America/New_York', 'Europe/London', 'Asia/Tokyo')
    
    Returns:
        Current datetime in the specified timezone
    
   

## 🤖 Crear Agente con Herramientas de Fecha/Hora

In [3]:
instructions = """
You are a helpful assistant with access to date and time tools.

You can:
- Tell users the current date and time
- Provide time in different timezones
- Get day of week, timestamps, etc.

Be friendly and format your responses nicely.
"""

request = "What's today's date and what day of the week is it?"
model = "gpt-4o-mini"

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(
        name="datetime_assistant",
        instructions=instructions,
        model=model,
        mcp_servers=[mcp_server]
    )
    
    with trace("datetime_query"):
        result = await Runner.run(agent, request)
    
    print("\n" + "="*60)
    print("💬 Agent Response:")
    print("="*60 + "\n")
    display(Markdown(result.final_output))


💬 Agent Response:



Today's date is **February 4, 2026**, and it is a **Wednesday**. If you need anything else, feel free to ask!

## 🧪 Test: Múltiples Consultas

In [4]:
test_queries = [
    "What time is it right now?",
    "Tell me the current date in ISO format",
    "What time is it in Tokyo right now?",
    "Give me the Unix timestamp for right now",
    "What's the current date and time in New York?"
]

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(
        name="datetime_assistant",
        instructions=instructions,
        model=model,
        mcp_servers=[mcp_server]
    )
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n{'='*60}")
        print(f"❓ Query {i}: {query}")
        print(f"{'='*60}\n")
        
        with trace(f"query_{i}"):
            result = await Runner.run(agent, query)
        
        display(Markdown(f"**Answer:** {result.final_output}"))
        print()


❓ Query 1: What time is it right now?



**Answer:** The current time is **10:24:45**. If you need the time in a specific timezone or have any other questions, feel free to ask!



❓ Query 2: Tell me the current date in ISO format



**Answer:** The current date in ISO format is **2026-02-04T10:24:47.880226**. If you need anything else, feel free to ask!



❓ Query 3: What time is it in Tokyo right now?



**Answer:** The current time in Tokyo is **6:24 PM JST**. If you need any more information, feel free to ask!



❓ Query 4: Give me the Unix timestamp for right now



**Answer:** The current Unix timestamp is **1770197094**.



❓ Query 5: What's the current date and time in New York?



**Answer:** The current date and time in New York is **February 4, 2026 at 04:24:56 EST**. If you need any other information, feel free to ask!

## 🌍 Test: Comparación de Múltiples Timezones

In [5]:
complex_request = """
I need to schedule an international meeting. Can you tell me:
1. The current date and time
2. What time it is right now in:
   - New York
   - London  
   - Tokyo
   - Sydney
3. What day of the week it is

Format it nicely so I can see all the times at once.
"""

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(
        name="datetime_assistant",
        instructions=instructions,
        model=model,
        mcp_servers=[mcp_server]
    )
    
    print("🌍 International Meeting Scheduler")
    print("="*60 + "\n")
    
    with trace("complex_datetime_query"):
        result = await Runner.run(agent, complex_request)
    
    display(Markdown(result.final_output))

🌍 International Meeting Scheduler



Here's the current date and time information formatted for your international meeting:

### Current Date and Time
- **UTC (Coordinated Universal Time)**: **Wednesday, February 04, 2026 at 10:25:48 AM**

### Time in Different Locations
| Location        | Time                    |
|-----------------|-------------------------|
| **New York**    | 2026-02-04 04:25:48 EST |
| **London**      | 2026-02-04 09:25:48 GMT |
| **Tokyo**       | 2026-02-04 18:25:48 JST |
| **Sydney**      | 2026-02-04 20:25:48 AEDT |

### Day of the Week
- **Today is**: **Wednesday**

If you need further assistance with scheduling, just let me know!

## 💡 Test: Conversación Contextual

In [6]:
# Test de conversación con contexto
conversation = [
    "What's today's date?",
    "And what time is it in London?",
    "How many hours difference is that from Tokyo?"
]

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(
        name="datetime_assistant",
        instructions=instructions,
        model=model,
        mcp_servers=[mcp_server]
    )
    
    print("💬 Conversational Test")
    print("="*60 + "\n")
    
    for message in conversation:
        print(f"👤 User: {message}")
        
        result = await Runner.run(agent, message)
        
        print(f"🤖 Agent: {result.final_output}")
        print()

💬 Conversational Test

👤 User: What's today's date?
🤖 Agent: Today's date is **February 4, 2026**. If you need more information or something else, just let me know!

👤 User: And what time is it in London?
🤖 Agent: The current time in London is **09:26 AM GMT** on February 4, 2026. If you need more information or help, feel free to ask!

👤 User: How many hours difference is that from Tokyo?
🤖 Agent: To provide an accurate time difference between Tokyo and your location, I need to know your current timezone or location. Could you please specify that?



## 📊 Resumen de Pruebas

Este notebook demostró:

✅ Cómo listar herramientas MCP disponibles  
✅ Crear un agente con acceso a herramientas MCP  
✅ Ejecutar consultas simples  
✅ Manejar consultas complejas multi-timezone  
✅ Mantener contexto en conversaciones  

### Próximos Pasos:

1. Ver `test_datetime_native.ipynb` para implementación sin Agent SDK
2. Modificar `datetime_server.py` para añadir más herramientas
3. Explorar otros MCP Servers públicos